### Convolutional Neural Network

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns


# Seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

### Plot formatting wrt revtex4 in LaTeX

In [ ]:
import matplotlib as mpl

# Widths in inches from revtex4's layout
# Single column ~3.375in, double column ~7in
columnwidth = 3.375  # use 7.0 for two-column-wide figures

# Compute figure size (width, height)
fig_width = columnwidth
fig_height = columnwidth / 1.618  # golden ratio for aesthetics
fig_size = [fig_width, fig_height]

mpl.rcParams.update({
    # Use LaTeX for text rendering
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": [],  # empty means use LaTeX default (Computer Modern)
    "font.size": 10.0,  # matches REVTeX's \normalsize

    # Adjust tick and label sizes
    "axes.labelsize": 10.0,
    "legend.fontsize": 8.0,
    "xtick.labelsize": 8.0,
    "ytick.labelsize": 8.0,

    # Figure dimensions
    "figure.figsize": fig_size,

    # Save with good resolution
    "savefig.dpi": 300,
})

Fetching data

In [ ]:
# Mean and standard deviation from data.ipynb
mean = [0.5]
std = [0.5]

# Resizing images to 480x480 pixels
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((480, 480)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)])

val_test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((480, 480)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)])

# Dataset
dataset_path = "chest_xray_data_split"
train_dataset = datasets.ImageFolder(f"{dataset_path}/train", transform=train_transform)
val_dataset   = datasets.ImageFolder(f"{dataset_path}/val", transform=val_test_transform)
test_dataset  = datasets.ImageFolder(f"{dataset_path}/test", transform=val_test_transform)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = train_dataset.classes


Model initialization

In [ ]:
# Parameters
num_classes = 2
num_epochs = 10
batch_size = 32 

# CNN model. Inspiration from https://www.datacamp.com/tutorial/pytorch-cnn-tutorial
class CNN(nn.Module):
    def __init__(self, padding=1, stride=2, kernel_size=3, pool_kernel=3, pool_stride=2):
        super().__init__()

        ## Convolutional layer
        # 1 input, 16 output features
        self.conv1 = nn.Conv2d(1, 16, kernel_size, stride, padding)
        # 16 input features, 32 output features
        self.conv2 = nn.Conv2d(16, 32, kernel_size, stride, padding)
        # 32 input features, 64 output features
        self.conv3 = nn.Conv2d(32, 64, kernel_size, stride, padding)

        # Pooling layer - same configuration for all pooling layers
        self.pool = nn.MaxPool2d(pool_kernel, pool_stride)

        # Calculates the size of first fully connected layer 
        with torch.no_grad():
            input = torch.zeros(1,1,480,480)
            feature1_size = self.pool(F.relu(self.conv1(input)))
            feature2_size = self.pool(F.relu(self.conv2(feature1_size)))
            feature3_size = self.pool(F.relu(self.conv3(feature2_size)))
            self.flatten_size = feature3_size.numel()

        # 2 Fully connected layer
        self.fc1 = nn.Linear(self.flatten_size, 32)
        self.fc2 = nn.Linear(32, num_classes-1)

    # Forward pass of the neural network.
    def forward(self, x):
        # x is the input tensor8
        x = F.relu(self.conv1(x)) # First convolutional layer with ReLU activation
        x = self.pool(x) # First max pooling layer
        x = F.relu(self.conv2(x)) # Second convolutionial layer with ReLU activation
        x = self.pool(x) # First max pooling layer
        x = F.relu(self.conv3(x)) # Third convolutionial layer with ReLU activation
        x = self.pool(x) # Third max pooling third
        x = x.view(x.size(0), -1) # flattening
        x = F.relu(self.fc1(x)) # fully connected layer
        return self.fc2(x)


Accuracy function

In [ ]:
def accuracy(logits, y):
    # 0.5 is the threshold value that determines what is classificed as a 1
    prediction = (torch.sigmoid(logits) >= 0.5).float()
    correct = (prediction == y).float().sum()
    return correct / y.numel()


Training function

In [ ]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim


def train_model(model, train_loader, val_loader, num_epochs=20, eta=1e-3,device="cpu"):

    model.to(device)
    cost_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=eta)

    # History lists for plotting
    train_cost = []
    val_cost = []
    train_acc = []
    val_acc = []

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        epoch_train_cost = 0
        epoch_train_acc  = 0

        # tqdm used to display estimated time per epoch
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            x = x.to(device)
            y = y.to(device).float().reshape(-1, 1)

            optimizer.zero_grad()
            logits = model(x)
            cost   = cost_fn(logits, y)

            cost.backward()
            optimizer.step()

            epoch_train_cost += cost.item()
            epoch_train_acc  += accuracy(logits, y).item()

        # Average training batch cost and accuracy
        avg_train_cost = epoch_train_cost / len(train_loader)
        avg_train_acc  = epoch_train_acc  / len(train_loader)

        # Validation data
        model.eval()
        epoch_val_cost = 0
        epoch_val_acc  = 0

        # Generating predictions on the validation data 
        with torch.no_grad():
            for x, y in val_loader:
                x = x.to(device)
                y = y.to(device).float().reshape(-1, 1)

                logits = model(x)
                cost   = cost_fn(logits, y)

                epoch_val_cost += cost.item()
                epoch_val_acc  += accuracy(logits, y).item()

        # Average validation batch cost and accuracy
        avg_val_cost = epoch_val_cost / len(val_loader)
        avg_val_acc  = epoch_val_acc  / len(val_loader)

        # Save history
        train_cost.append(avg_train_cost)
        val_cost.append(avg_val_cost)
        train_acc.append(avg_train_acc)
        val_acc.append(avg_val_acc)

        print(f"Epoch {epoch+1}/{num_epochs} Train cost: {avg_train_cost:.4f} Validation cost: {avg_val_cost:.4f} Validation accuracy: {avg_val_acc:.4f}")

    return {"train_cost": train_cost,
            "val_cost":   val_cost,
            "train_acc":  train_acc,
            "val_acc":    val_acc}


#### Hyperparameterizing filter size - convolutional layer

In [ ]:
# Hyperparameter
kernel_sizes = [2,3,4,5]

kernel_results = []
kernel_metrics = []

# Iterating over the hyperparameters
for size in kernel_sizes:
        # Initializing CNN with hyperparameters
        model = CNN(kernel_size=size)  
        # Metrics from trained model
        kernel_conv_layer = train_model(model, train_loader, val_loader, num_epochs=10)

        # Store metrics for the current hyperparameter combination
        kernel_metrics.append({"kernel_size": size, "history": kernel_conv_layer})

        # Evaluating the current hyperparameter combination
        model.eval()

        cost_accumulated = 0.0
        n_samples = 0
        
        y_true = []
        y_pred = []

        # Generating predictions on the test data 
        with torch.no_grad():
            for x, y in test_loader:
                logits = model(x)

                # BCEWithLogitsLoss expects float (1.0, 0.0)
                y = y.float().unsqueeze(1)

                current_cost = torch.nn.BCEWithLogitsLoss()(logits, y)

                cost_accumulated += current_cost.item() * x.size(0)
                n_samples += x.size(0)

                probs = torch.sigmoid(logits)
                preds = (probs >= 0.5).int()

                y_true.extend(y.squeeze(1).int().numpy())
                y_pred.extend(preds.numpy())

        # Average test_cost
        test_cost = cost_accumulated / n_samples

        # Calculating the TN, FP, FN and TP
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()

        # Calculating sensitivity and specificity
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)


        # Storing results
        kernel_results.append({
            "kernel_size": size,
            "test_cost": test_cost,
            "confusion_matrix": cm,
            "sensitivity": sensitivity,
            "specificity": specificity})

In [ ]:
kernel_sizes = sorted([r["kernel_size"] for r in kernel_results])
kernel_costs = [r["test_cost"] for r in kernel_results]

# Initialize matrices
cost_matrix_kernel = np.array(kernel_costs).reshape(1, -1)
combined_matrix = np.zeros((2, len(kernel_sizes)))

# Formatting labels to display "2x2", "3x3", ...
kernel_labels = [f"{k}x{k}" for k in kernel_sizes]

# Combining sensitivity and specificity results into one matrix for plotting
for j, r in enumerate(kernel_results):
    combined_matrix[0, j] = r["sensitivity"]
    combined_matrix[1, j] = r["specificity"]


#### Cost - Filter size

In [ ]:
plt.figure()
sns.heatmap(cost_matrix_kernel,annot=True,fmt=".4f",cmap="Greens",xticklabels=kernel_labels,yticklabels=[" "])
plt.xlabel("Filter Size")
plt.title("Binary Cross-Entropy - Conv layer")
plt.tight_layout()
plt.show()

#### Sensitivity & specificity - Filter size

In [ ]:
plt.figure()
sns.heatmap(combined_matrix,annot=True,fmt=".3f",cmap="Greens",xticklabels=kernel_labels,yticklabels=["Sensitivity", "Specificity"])
plt.xlabel("Filter Size")
plt.title("Sensitivity and Specificity")
plt.tight_layout()
plt.show()

### Hyperparameterizing padding and stride - Convolutional layer

In [ ]:
# Hyperparameters
padding = [0,1,2]
strides = [1,2,3]
epochs = 10

# Initialzing lists to store evaluation metrics across models
results = []
conv_metrics = []

# Iterating over the hyperparameters
for pad in padding:
    for stride in strides:
        # Initializing CNN with hyperparameters
        model = CNN(padding=pad, stride=stride)
        # Metrics from trained models
        convolutional_layer = train_model(model,train_loader,val_loader,num_epochs=epochs)
        # Store metrics for the current hyperparameter combination
        conv_metrics.append({"padding": pad,"stride": stride,"history": convolutional_layer})

        # Evaluate the trained model
        model.eval()
        
        cost_accumulated = 0.0
        n_samples = 0
        
        y_true = []
        y_pred = []


        # Generating predictions on the test data 
        with torch.no_grad():
            for x, y in test_loader:
                logits = model(x)

                # BCEWithLogitsLoss expects float (1.0, 0.0)
                y = y.float().unsqueeze(1)
                
                current_cost = torch.nn.BCEWithLogitsLoss()(logits, y)

                cost_accumulated += current_cost.item() * x.size(0)
                n_samples += x.size(0)

                probs = torch.sigmoid(logits)
                preds = (probs >= 0.5).int()

                y_true.extend(y.squeeze(1).int().numpy())
                y_pred.extend(preds.numpy())

        # Average test_cost
        test_cost = cost_accumulated / n_samples

        # Calculating the TN, FP, FN and TP
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()

        # Calculating sensitivity and specificity
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)

        # Storing results
        results.append({
            "padding": pad,
            "stride": stride,
            "test_cost": test_cost,
            "confusion_matrix": cm,
            "sensitivity": sensitivity,
            "specificity": specificity
            })


In [ ]:
# Generating labels
padding_values = sorted(set(r["padding"] for r in results))
stride_values = sorted(set(r["stride"] for r in results))

# Initialize matrices
cost_matrix_conv = np.zeros((len(padding_values), len(stride_values)))
sensitivity_matrix_conv = np.zeros_like(cost_matrix_conv)
specificity_matrix_conv = np.zeros_like(cost_matrix_conv)

# Fill matrices
for r in results:
    i = padding_values.index(r["padding"])
    j = stride_values.index(r["stride"])
    
    cost_matrix_conv[i, j] = r["test_cost"] if "test_cost" in r else np.nan
    sensitivity_matrix_conv[i, j] = r["sensitivity"]
    specificity_matrix_conv[i, j] = r["specificity"]


#### Cost Analysis - Convolutional Layer

In [ ]:
plt.figure()
sns.heatmap(cost_matrix_conv,annot=True,fmt=".4f",xticklabels=stride_values,yticklabels=padding_values,cmap="Greens")
plt.xlabel("Stride")
plt.ylabel("Padding")
plt.title("Binary Cross-Entropy - Conv Layer")
plt.tight_layout()
plt.show()

#### Sensitivity Analysis - Convolutional Layer

In [ ]:
plt.figure()
sns.heatmap(sensitivity_matrix_conv,annot=True,fmt=".3f",xticklabels=stride_values,yticklabels=padding_values,cmap="Greens")
plt.xlabel("Stride")
plt.ylabel("Padding")
plt.title("Sensitivity - Conv Layer")
plt.tight_layout()
plt.show()

#### Specitivity Analysis - Convolutional Layer

In [ ]:
plt.figure()
sns.heatmap(specificity_matrix_conv,annot=True,fmt=".3f",xticklabels=stride_values,yticklabels=padding_values,cmap="Greens")
plt.xlabel("Stride")
plt.ylabel("Padding")
plt.title("Specificity - Conv Layer")
plt.tight_layout()
plt.show()


### Hyperparameterizing max pooling kernel size and stride

In [ ]:
# Hyperparameters
pooling_kernel_sizes = [2,3,4,5]
pooling_strides = [1,2,3,4]

# Initialzing lists to store evaluation metrics across models
pooling_results = []
pooling_metrics = []

# Iterating over the hyperparameters
for pool_kernel in pooling_kernel_sizes:
    for pool_stride in pooling_strides:


        # Initializing CNN with hyperparameters
        model = CNN(padding = 1, stride = 2, pool_kernel=pool_kernel, pool_stride=pool_stride)  
        # Metrics from trained model
        pooling_layer = train_model(model, train_loader, val_loader, num_epochs=10)

        # Store metrics for the current hyperparameter combination
        pooling_metrics.append({"pool_kernel": pool_kernel,"pool_stride": pool_stride, "history": pooling_layer})

        # Evaluating the current hyperparameter combination
        model.eval()

        cost_accumulated = 0.0
        n_samples = 0
        y_true = []
        y_pred = []

        # Generating predictions on the test data 
        with torch.no_grad():
            for x, y in test_loader:
                logits = model(x)

                # BCEWithLogitsLoss expects float 1, 0 --> 1.0, 0.0
                y = y.float().unsqueeze(1)

                current_cost = torch.nn.BCEWithLogitsLoss()(logits, y)

                cost_accumulated += current_cost.item() * x.size(0)
                n_samples += x.size(0)

                probs = torch.sigmoid(logits)
                preds = (probs >= 0.5).int()

                y_true.extend(y.squeeze(1).int().numpy())
                y_pred.extend(preds.numpy())

        # Average test_cost
        test_cost = cost_accumulated / n_samples

        # Calculating the TN, FP, FN and TP
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()

        # Calculating sensitivity and specificity
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)


        # Storing results
        pooling_results.append({
            "pool_kernel": pool_kernel,
            "pool_stride": pool_stride,
            "test_cost": test_cost,
            "confusion_matrix": cm,
            "sensitivity": sensitivity,
            "specificity": specificity
        })



In [ ]:
# Generating labels
pool_kernel_values = sorted(set(r["pool_kernel"] for r in pooling_results))
pool_stride_values = sorted(set(r["pool_stride"] for r in pooling_results))

# Formatting labels as "2x2", "3x3", ..., "NxN"
pool_kernel_labels = [f"{k}x{k}" for k in pool_kernel_values]


# Initialize matrices
cost_matrix_pooling = np.full((len(pool_kernel_values), len(pool_stride_values)), np.nan)
sensitivity_matrix_pooling = np.full_like(cost_matrix_pooling, np.nan)
specificity_matrix_pooling = np.full_like(cost_matrix_pooling, np.nan)

# Fill matrices
for r in pooling_results:
    i = pool_kernel_values.index(r["pool_kernel"])
    j = pool_stride_values.index(r["pool_stride"])

    cost_matrix_pooling[i, j] = r["test_cost"]
    sensitivity_matrix_pooling[i, j] = r["sensitivity"]
    specificity_matrix_pooling[i, j] = r["specificity"]


#### Cost Analysis - Pooling Layer

In [ ]:
plt.figure()
sns.heatmap(cost_matrix_pooling,annot=True,fmt=".4f",xticklabels=pool_stride_values, yticklabels=pool_kernel_labels,cmap="Greens")
plt.xlabel("Filter Stride")
plt.ylabel("Filter Size")
plt.title("Binary Cross-Entropy - Pooling Layer")
plt.tight_layout()
plt.show()

#### Sensitivity Analysis - Pooling Layer

In [ ]:
plt.figure()
sns.heatmap(sensitivity_matrix_pooling,annot=True,fmt=".3f",xticklabels=pool_stride_values,yticklabels=pool_kernel_labels,cmap="Greens")
plt.xlabel("Filter Stride")
plt.ylabel("Filter Size")
plt.title("Sensitivity - Pooling Layer")
plt.tight_layout()
plt.show()

#### Specitivity Analysis - Pooling Layer

In [ ]:
plt.figure()
sns.heatmap(specificity_matrix_pooling,annot=True,fmt=".3f",xticklabels=pool_stride_values,yticklabels=pool_kernel_labels,cmap="Greens")
plt.xlabel("Filter Stride")
plt.ylabel("Filter Size")
plt.title("Specificity - Pooling Layer")
plt.tight_layout()
plt.show()

### Final model

In [ ]:
# Final parameters
## Convolutional layer
kernel_size_conv = 3
stride_conv = 3
padding_conv = 3


## Pooling layer
kernel_size_pooling = 3
stride_pooling = 1

final_results = []
final_metrics = []

# Initializing CNN with hyperparameters
model = CNN(kernel_size= kernel_size_conv, padding = padding_conv, stride = stride_conv, pool_kernel=kernel_size_pooling, pool_stride=stride_pooling)  
# Metrics from trained model
final = train_model(model, train_loader, val_loader, num_epochs=15)

# Store metrics for the current hyperparameter combination
final_metrics.append({"history": final})

# Evaluating the current hyperparameter combination
model.eval()

cost_accumulated = 0.0
n_samples = 0
y_true = []
y_pred = []

# Generating predictions on the test data 
with torch.no_grad():
    for x, y in test_loader:
        logits = model(x)

        # BCEWithLogitsLoss expects float (1.0, 0.0)
        y = y.float().unsqueeze(1)

        current_cost = torch.nn.BCEWithLogitsLoss()(logits, y)

        cost_accumulated += current_cost.item() * x.size(0)
        n_samples += x.size(0)

        probs = torch.sigmoid(logits)
        preds = (probs >= 0.5).int()

        y_true.extend(y.squeeze(1).int().numpy())
        y_pred.extend(preds.numpy())

# Average test_cost
test_cost = cost_accumulated / n_samples

# Calculating the TN, FP, FN and TP
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()

# Calculating sensitivity and specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)


# Storing results
final_results.append({
    "test_cost": test_cost,
    "confusion_matrix": cm,
    "sensitivity": sensitivity,
    "specificity": specificity
})



#### Confusion Matrix, Performance Metrics

In [ ]:
# Accuracy
accuracy = (tp + tn) / (tp + tn + fp + fn)

# Printing metrics

print(f"Accuracy: {accuracy:.4f}")
print(f"Test Cost: {test_cost:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")


plt.figure(figsize=(3.5, 2.5))
ax = sns.heatmap(cm,annot=True,fmt="d",cmap="Greens",xticklabels=["No Pneumonia", "Pneumonia"],yticklabels=["No Pneumonia", "Pneumonia"])
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')
ax.invert_yaxis()
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()


#### Model summary

In [ ]:
from torchinfo import summary
# (batch, channels, height, width)
summary(model,input_size=(32, 1,480, 480),col_names=["input_size", "output_size", "num_params", "kernel_size"],depth=4)


#### Extracting feature maps from the convolutional layers

In [ ]:
# Empty dictionary to store feature maps from the convolutional layers
feature_maps = {}


def make_hook(name):
    def hook(module, input, output):
        feature_maps[name] = output.detach()
    return hook

h1 = model.conv1.register_forward_hook(make_hook("conv1"))
h2 = model.conv2.register_forward_hook(make_hook("conv2"))
h3 = model.conv3.register_forward_hook(make_hook("conv3"))

# Forward pass
model.eval()
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(next(model.parameters()).device)
        _ = model(x)
        break

# Removing hook
h1.remove()
h2.remove()
h3.remove()

# Image index
image = 9
feature_idx = [0, 1, 2, 3]

layers = ["conv1", "conv2", "conv3"]

fig, axes = plt.subplots(len(layers), len(feature_idx), figsize=(4, 4))

for row, layer in enumerate(layers):
    fmap = feature_maps[layer][image] 

    for col, feature_index in enumerate(feature_idx):
        fm = fmap[feature_index].cpu()
        fm = (fm - fm.min()) / (fm.max() - fm.min() + 1e-8)

        ax = axes[row, col]
        ax.imshow(fm, cmap="viridis")

        H, W = fm.shape

        ax.set_xticks([0, W//2, W-1])
        ax.set_yticks([0, H//2, H-1])

        ax.set_xticklabels([0, W//2, W-1], fontsize=7)
        ax.set_yticklabels([0, H//2, H-1], fontsize=7)


        if row == 0:
            ax.set_title(f"Feature {feature_index}", fontsize=9)


    axes[row, 0].set_ylabel(layer, fontsize=11, rotation=90, labelpad=3)

    
plt.subplots_adjust(wspace=0.4,   hspace=0.000)
plt.suptitle("Feature Maps Across Convolutional Layers", fontsize=12)
plt.tight_layout
plt.show()


#### Overlaying features on the input image

In [ ]:
# Function that finds two images with (prob>0.9) and without pneumonia (prob<0.1). 
def image_search(model, test_loader, device="cpu",pos_thresh=0.9, neg_thresh=0.1):
    model.eval()
    model.to(device)

    pos_case = None
    neg_case = None

    # Forward pass with predictions
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            logits = model(x)               
            probs = torch.sigmoid(logits)    

            for i in range(x.size(0)):
                prob = probs[i].item()
                label = y[i].item()

                if pos_case is None and label == 1 and prob >= pos_thresh:
                    pos_case = {"img": x[i:i+1].cpu(),"label": int(label),"logit": logits[i].item(),"prob": prob}

                if neg_case is None and label == 0 and prob <= neg_thresh:
                    neg_case = {"img": x[i:i+1].cpu(),"label": int(label),"logit": logits[i].item(),"prob": prob}

                if pos_case is not None and neg_case is not None:
                    return pos_case, neg_case

    return pos_case, neg_case

# Finding two images with and without pneumonia
pos_case, neg_case = image_search(model,test_loader,device="cpu",pos_thresh=0.8,neg_thresh=0.1)

fig, axes = plt.subplots(1, 2)

img  = pos_case["img"]
prob = pos_case["prob"]

cam = gradcam(model, img, conv_layer=model.conv3)

xray = img.squeeze()
xray = (xray - xray.min()) / (xray.max() - xray.min() + 1e-8)

axes[0].imshow(xray, cmap="gray")
axes[0].imshow(cam, alpha=0.3)
axes[0].axis("off")
axes[0].set_title(f"Pneumonia\nProbability: {prob:.2f}")


img  = neg_case["img"]
prob = neg_case["prob"]

cam = gradcam(model, img, conv_layer=model.conv3)

xray = img.squeeze()
xray = (xray - xray.min()) / (xray.max() - xray.min() + 1e-8)

axes[1].imshow(xray, cmap="gray")
axes[1].imshow(cam, alpha=0.3)
axes[1].axis("off")
axes[1].set_title(f"No Pneumonia\nProbability: {prob:.2f}")

plt.tight_layout()
plt.show()


